# The Stepsize of DSEA+

DSEA+ extends the original DSEA with an adaptively chosen stepsize between iterations. This tutorial assumes you already know the other notebook at `doc/01-getting-started.ipynb`.

In [1]:
using CherenkovDeconvolution
using ScikitLearn, MLDataUtils, Random
using Discretizers: encode, CategoricalDiscretizer

# load the example data, encode labels with integers
X, y_labels, _ = load_iris()
y = encode(CategoricalDiscretizer(y_labels), y_labels)

# split the data into training and observed data sets
Random.seed!(42) # make split reproducible
(X_train, y_train), (X_data, y_data) = splitobs(shuffleobs((X', y), obsdim = 1), obsdim = 1)

# discretize the feature space
td = Binnings.TreeDiscretizer(X_train, y_train, 3) # obtain up to 3 clusters
x_train = encode(td, X_train)
x_data  = encode(td, X_data)

# also prepare the classifier for DSEA
@sk_import naive_bayes : GaussianNB
tp_function = Binnings.train_and_predict_proba(GaussianNB());

## Adaptive Step Size

The adaptive step size is specified through the `alpha` argument of DSEA. This argument expects a `CherenkovDeconvolution.Stepsize` object, for which CherenkovDeconvolution.jl provides several implementations.

The most important implementation, `RunStepsize` uses the objective function of the regularized unfolding (RUN) to determine the step size adaptively. We further specify `epsilon`, the minimum Chi square distance between iterations. Convergence is assumed if the distance drops below this threshold.

In this example, convergence is assumed immediately because the input- and output distributions are approximately equal.

In [2]:
alpha = RunStepsize(x_data, x_train, y_train; decay=true)
f_dsea = dsea(X_data, X_train, y_train, tp_function, K=100, epsilon=1e-6, alpha=alpha)

┌ Info: DSEA iteration 1/100 uses alpha = 2.828411083473901e-13 (chi2s = 2.237160223400213e-28)
└ @ CherenkovDeconvolution /home/bunse/.julia/dev/CherenkovDeconvolution/src/methods/dsea.jl:150
┌ Info: DSEA convergence assumed from chi2s = 2.237160223400213e-28 < epsilon = 1.0e-6
└ @ CherenkovDeconvolution /home/bunse/.julia/dev/CherenkovDeconvolution/src/methods/dsea.jl:155


3-element Array{Float64,1}:
 0.3333333333333333
 0.3333333333333394
 0.3333333333333272

## Other Step Sizes

Another adaptive step size, based on a least-square objective, is the `LsqStepsize`. Two decaying step sizes can be obtained with `ExpDecayStepsize` and `MulDecayStepsize`. There is also a `ConstantStepsize`.

If you want to implement additional step size strategies, you only need an implementation of the `stepsize` method for a custom `Stepsize` type.

## Further Documentation

In [3]:
?stepsize

search: stepsize Stepsize RunStepsize LsqStepsize MulDecayStepsize



```
stepsize(s, k, p, f, a)
```

Use the `Stepsize` object `s` to compute a step size for iteration number `k` with the search direction `p`, the previous estimate `f`, and the previous step size `a`.

**See also:** `ConstantStepsize`, `RunStepsize`, `LsqStepsize`, `ExpDecayStepsize`, `MulDecayStepsize`.


In [4]:
?RunStepsize

search: RunStepsize



```
RunStepsize(x_data, x_train, y_train[, tau=0; bins_y, bins_x, warn=false, decay=false])
```

Adapt the step size by maximizing the likelihood of the next estimate in the search direction of the current iteration.

The arguments of this function reflect a discretized deconvolution problem, as used in RUN. Setting `decay=true` will enforce that a*k+1 <= a*k, i.e. the step sizes never increase.

**See also:** `OptimizedStepsize`.


In [5]:
?LsqStepsize

search: LsqStepsize



```
LsqStepsize(x_data, x_train, y_train[, tau=0; bins_y, bins_x, warn=false, decay=false])
```

Adapt the step size by solving a least squares objective in the search direction of the current iteration.

The arguments of this function reflect a discretized deconvolution problem, as used in RUN. Setting `decay=true` will enforce that a*k+1 <= a*k, i.e. the step sizes never increase.

**See also:** `OptimizedStepsize`.


In [6]:
?ExpDecayStepsize

search: ExpDecayStepsize



```
ExpDecayStepsize(eta, a=1.0)
```

Reduce the first stepsize `a` by `eta` in each iteration:

```
stepsize(ExpDecayStepsize(eta, a), k, ...) == a * eta^(k-1)
```


In [7]:
?MulDecayStepsize

search: MulDecayStepsize



```
MulDecayStepsize(eta, a=1.0)
```

Reduce the first stepsize `a` by `eta` in each iteration:

```
stepsize(MulDecayStepsize(eta, a), k, ...) == a * k^(eta-1)
```


In [8]:
?ConstantStepsize

search: ConstantStepsize



```
ConstantStepsize(alpha)
```

Choose the constant step size `alpha` in every iteration.
